# 3D printing with Jubilee
Let's start by initiating communication with the machine and define the gel extruder(s).

In [2]:
from science_jubilee.Machine import Machine
from science_jubilee.tools.Tool import Tool
from science_jubilee.tools.SyringeExtruder import SyringeExtruder

In [3]:
m = Machine(address = "192.168.1.2")
m

/Users/danliluo/HCDE/Dubilee/science_jubilee/science_jubilee/Machine.py:332: UserWarning: Connection initiated with tool equipped. Use reload_tool() after instantiate this tool.
  warnings.warn("Connection initiated with tool equipped. Use reload_tool() after instantiate this tool.")


The first thing you do is to home the machine!

In [ ]:
m.home_all()

Homing could take a minute. Now, define and load tool(s).

In [5]:
syringe_0 = SyringeExtruder(0, "white syringe")
syringe_1 = SyringeExtruder(1, "orange syringe")
m.reload_tool(syringe_0) # if you reinitiate a tool, use reload_tool instead of load_tool
m.load_tool(syringe_1)
m.tools

{0: {'name': 'white syringe',
  'tool': <science_jubilee.tools.SyringeExtruder.SyringeExtruder at 0x7f98f0987370>},
 1: {'name': 'orange syringe',
  'tool': <science_jubilee.tools.SyringeExtruder.SyringeExtruder at 0x7f98f09878b0>}}

3D printing fine structure is very sensitive to the z offset. Z offset changes with the nozzle you're using, the length of the syringe, etc. Now we need to zero the nozzle tip and update the z offset.

First, pick up the tool you'd like to calibrate.

In [6]:
m.pickup_tool(0)

The z offset on startup is an overshoot to prevent tool crashing. Let's move to the current `Z = 0`.

In [7]:
m.move_to(z = 0)

Science Jubilee would prevent you from going further! Use the method `approach` inside `SyringeExtruder` to bring the tip of the syringe extruder into contact with the print bed. The z offset is automatically updated with `approach` as you do so.

In [29]:
syringe_0.approach(-0.01) # you may need to approach many times to get the right position
print(m.tool_z_offsets[0])

-26.31


If it says `"MachineStateError: Error: Relative move exceeds Z axis limit!"`, temporarily update the z offset to allow the nozzle to go lower. 

Before loading your slicer-generated gcode file, it's a good practice to prime the nozzle - extrude a little bit of material so that the nozzle is filled and ready to print.

Noted that the parameter of `extrude` is the length of the filament, i.e. the actual plunger movement. 

In [ ]:
m.move(dz = 25) # move the nozzle up if it's too close to the bed
syringe_0.extrude(0.1) # you may need to extrude many times. An empty tapered nozzle needs ~1.5 to fill up

Helper functions to load gcode and print gcode:

In [ ]:
# load the .gcode file and parse
def load_gcode(file_path):
    try:
        lines = []
        with open(file_path, 'r') as file:
            for line in file:
                lines.append(line.strip())
        return lines
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def print_gcode(gcode):
    for line in gcode:
        if len(line) > 0:
            if not line.startswith(';'):
                print(line)
                m.gcode(line)


In [ ]:
gcode = load_gcode("cylinder-20mm.gcode")
print_gcode(gcode)

In [31]:
m.park_tool()